In [5]:
import requests
from bs4 import BeautifulSoup
import json
import unicodedata
import re
from unidecode import unidecode
import os
import unicodedata

In [17]:
def extract_movie_links():
    url = "https://www.athinorama.gr/cinema/guide/"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find all div elements with class "item horizontal card-item"
    movie_cards = soup.find_all('div', class_='item horizontal card-item')
    movie_links = []
    for card in movie_cards:
        # Find the link inside item-title div
        title_div = card.find('h2', class_='item-title')
        if title_div:
            link = title_div.find('a')
            if link and link.get('href'):
                movie_links.append(link['href'])
    
    return movie_links


def get_movie_times(url):
    resp = requests.get(url)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    # Find all inner-panel divs
    panels = soup.find_all("div", class_="panel-inner")

    results = []
    for panel in panels:
        # Get all elements with class daytimeschedule inside each inner-panel
        schedules = panel.find_all(class_="daytimeschedule")
        times = [s.get_text(strip=True) for s in schedules]
        if times:
            results.append(times)
    # Print extracted schedules
    for i, sched in enumerate(results, 1):
        print(f"Panel {i}: {sched}")
        

def get_movie_theater(url):

    # fetch the page
    response = requests.get(url)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, "html.parser")

    # find all theater blocks
    theaters = []
    for title_tag, details_tag in zip(
        soup.find_all("h2", class_="item-title"),
        soup.find_all("div", class_="details")):
        name = title_tag.get_text(strip=True)
        address = details_tag.get_text(" ", strip=True)  # keep spacing
        theaters.append({"name": name, "address": address})

    # print results
    for t in theaters:
        print(f"{t['name']} - {t['address']}")



def is_greek(text):
    """Return True if text contains mostly Greek characters."""
    greek_chars = re.findall(r"[Α-Ωα-ωάέήίόύώΆΈΉΊΌΎΏ]", text)
    return len(greek_chars) > len(text) * 0.5  # >50% Greek letters = Greek text

def transliterate_greek_to_latin(text):
    """Convert Greek to Latin using unidecode if needed."""
    if is_greek(text):
        return unidecode(text)
    return text


def get_cinema_info_from_google(name: str, address: str = None):
    """Fetch cinema info (lat, lon, area, formatted address) from Google Maps API."""
    query = name if not address else f"{name}, {address}"
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        "address": query,
        "key": GOOGLE_API_KEY,
        "language": "el"  # or "en" depending on what you want
    }

    response = requests.get(url, params=params)
    data = response.json()

    if data["status"] != "OK" or not data["results"]:
        print(f"⚠️ Google Maps API: No match for '{query}'")
        return {
            "lat": None,
            "lon": None,
            "area": "Unknown",
            "formatted_address": None
        }

    result = data["results"][0]
    geometry = result["geometry"]["location"]
    address_components = result.get("address_components", [])

    # Try to extract area (e.g., neighborhood, locality, sublocality)
    area = "Unknown"
    # Extract broader area (default: locality)
    area = next(
        (c["long_name"] for c in address_components if "locality" in c["types"]),
        "Unknown"
    )

    formatted_addr = result.get("formatted_address")
    
    # 🧹 Step 3: Remove Greek street words and abbreviations
    first_part = re.sub(
        r'\b(Λ\.?|Λεωφόρος|Λεωφ\.?|Οδός|Οδ\.?|Δρόμος|Δρ\.?)\b',
        '',
        formatted_addr,
        flags=re.IGNORECASE
    ).strip()

    # 🧹 Step 4: Keep only up to the first '&' or 'και' or '-' (e.g., "Συγγρού & Φραντζή" → "Συγγρού")
    first_part = re.split(r'\s*&\s*|\s*και\s*|\s*-\s*', first_part)[0].strip()
    
    # --- Geocoding ---
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": f"{first_part}",
        "format": "json",
        "addressdetails": 1,
        "limit": 1
    }

    
    r = requests.get(url, params=params, headers={"User-Agent": "cinema-app"})
    r.raise_for_status()
    data = r.json()
    if data:
        details = data[0].get("address", {})
        open_info_suburb = details.get("suburb")
        open_info_neighbourhood = details.get("neighbourhood")
    else:
        open_info_suburb = open_info_neighbourhood = ""
        
    

    return {
        "lat": geometry["lat"],
        "lon": geometry["lng"],
        "area": area,
        "suburb": open_info_suburb,
        "neighbourhood": open_info_neighbourhood,
        "formatted_address": formatted_addr
        }


def geocode_area(address):
    # 🧹 Step 1: Remove parentheses and contents inside them
    cleaned = re.sub(r'\([^)]*\)', '', address).strip()

    # 🧹 Step 2: Keep only the first comma-separated part
    # first_part = cleaned.split(',')[0].strip()

    # 🧹 Step 3: Remove Greek street words and abbreviations
    first_part = re.sub(
        r'\b(Λ\.?|Λεωφόρος|Λεωφ\.?|Οδός|Οδ\.?|Δρόμος|Δρ\.?)\b',
        '',
        first_part,
        flags=re.IGNORECASE
    ).strip()

    # 🧹 Step 4: Keep only up to the first '&' or 'και' or '-' (e.g., "Συγγρού & Φραντζή" → "Συγγρού")
    first_part = re.split(r'\s*&\s*|\s*και\s*|\s*-\s*', first_part)[0].strip()

#     # 🧹 Step 5: Keep only first word and possible number (e.g. "Παπανδρέου 12")
#     match = re.match(r'^([\wΆ-ώΑ-Ωά-ώ]+(?:\s*\d{1,3})?)', first_part)
#     if match:
#         query_base = match.group(1)
#     else:
#         query_base = first_part

    # 🧹 Step 6: Collapse spaces
    query_base = re.sub(r'\s+', ' ', query_base).strip()

    # --- Geocoding ---
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": f"{query_base}",
        "format": "json",
        "addressdetails": 1,
        "limit": 1
    }

#     print(f"Geocoding query: {params['q']}")

    try:
        r = requests.get(url, params=params, headers={"User-Agent": "cinema-app"})
        r.raise_for_status()
        data = r.json()
        if not data:
            return None

        details = data[0].get("address", {})
        area = (
            details.get("suburb")
            or details.get("neighbourhood")
            or details.get("city_district")
            or details.get("town")
            or details.get("city")
        )

        lat = data[0].get("lat")
        lon = data[0].get("lon")

        return {"area": area, "lat": lat, "lng": lon}

    except Exception as e:
        print(f"Error geocoding {address}: {e}")
        return None


def normalize_name(name: str) -> str:
    """Normalize and clean cinema name for reliable matching."""
    if not name:
        return ""
    # Normalize Unicode (remove accent inconsistencies)
    name = unicodedata.normalize("NFKC", name)
    # Remove invisible spaces, trim, and lowercase
    name = name.strip().replace("\u200b", "").replace("\xa0", " ").lower()
    return name
    

def get_movie_theater_times(url):
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")
    movies_data = []
    cinemas_data = []
    
    # --- Movie Titles ---
    title_greek_tag = soup.find("h1")
    title_greek = title_greek_tag.get_text(strip=True) if title_greek_tag else "Unknown Title"

            
    card = soup.find_all("ul", class_="review-details")
    for c in card:
        original_tag = c.find("span", class_="original-title")
        if original_tag:
            original_title = original_tag.get_text()
        else:
            original_title = ""
            
    imdb = soup.find('a',class_="imdb")
    imdb = imdb.get('href') if imdb else None
        

    movies_data.append({
        "greek_title": title_greek,
        "original_title": original_title,
        "athinorama_link": url,
        "imdb_link": imdb
    })

    # --- Cinema Entries ---
    cinema_blocks = soup.find_all("div", class_="item card-item")
    for block in cinema_blocks:
        name_tag = block.find("h2", class_="item-title")
        details_tag = block.find("div", class_="details")
        name = name_tag.get_text(strip=True) if name_tag else None

        # Rooms
        rooms = []
        for panel in block.find_all("div", class_="grid schedule-grid"):
            room_name_tag = panel.find("span")
            room_name = room_name_tag.get_text(strip=True) if room_name_tag else "Main Room"
            rooms.append({"room": room_name})

        # Timetable
        room_timetable = []
        innerpanels = block.find_all("div", class_="panel-inner")
        for panel in innerpanels:
            schedules = panel.find_all(class_="daytimeschedule")
            times = [s.get_text(strip=True) for s in schedules]
            if times:
                room_timetable.append(times)
                
        # --- Normalize name before matching ---
        norm_name = normalize_name(name)
        existing_cinema = None
#         existing_cinema = next(
#         (c for c in cinemas_data if normalize_name(c["cinema"]) == norm_name),
#             None
#         )

        if existing_cinema:
            # ✅ Update only the timetable
            print(name, 'cinema info exists')
            existing_cinema["timetable"] = room_timetable
            existing_cinema["rooms"] = rooms
        else:
            print(name, 'did not find cinema info')
            address = details_tag.get_text(" ", strip=True) if details_tag else None
            region_dict = get_cinema_info_from_google(name, address)
            if not region_dict:
                region_dict['area'] = 'Unknown'
                region_dict['subarea'] = 'Unknown'
                region_dict['formatted_address'] = address
                region_dict['lat'] = 0
                region_dict['lon'] = 0

            cinemas_data.append({
                "cinema": name,
                "address": region_dict['formatted_address'],
                "region": region_dict['area'],
                "subregion": region_dict['suburb'],
                "neighbourhood": region_dict['neighbourhood'],
                "rooms": rooms,
                "timetable": room_timetable
            })

    return movies_data, cinemas_data

In [7]:
movie_links =[]

base_url = "https://www.athinorama.gr"
if __name__ == "__main__":
    links = extract_movie_links()
    for link in links:
        print(link)
        movie_links.append(base_url + link)  

/cinema/movie/to_47-10086638
/cinema/movie/agapi_mono-10087322
/cinema/movie/alpeis-10009258
/cinema/movie/amugdalia-10068525
/cinema/movie/o_anthropos_me_to_xruso_xeri-10048555
/cinema/movie/o_anthropos_xoris_parelthon-1006824
/cinema/movie/o_astakos-10049281
/cinema/movie/axinos-10087613
/cinema/movie/bampir-10032849
/cinema/movie/good_boy-10087321
/cinema/movie/i_gunaika_me_ta_spirta-1004219
/cinema/movie/ena_aplo_atuxima-10087413
/cinema/movie/eksolothreutis_aggelos-1007248
/cinema/movie/erotas_sti_barkeloni-10087053
/cinema/movie/f1_i_tainia_-10086559
/cinema/movie/efialtis_sto_dromo_me_tis_leukes-1000080
/cinema/movie/o_thanatos_tou_ierou_elafiou-10058117
/cinema/movie/interstellar-10040927
/cinema/movie/istories_tou_sikouana-10087194
/cinema/movie/kaiti_drosou_kibotio_mnimis-10087669
/cinema/movie/ta_kaka_paidia_2-10086927
/cinema/movie/kales_giortes-10087398
/cinema/movie/to_kalesma_4_teleutaia_teletourgia-10086923
/cinema/movie/i_katara_tou_selmpi_oouks-10087454
/cinema/movie/

In [21]:
movies_l = []
cinemas_l = []

# # --- Load existing cinema data ---
# cinema_file = "cinemas.json"

# if os.path.exists(cinema_file):
#     with open(cinema_file, "r", encoding="utf-8") as f:
#         try:
#             cinemas_data = json.load(f)
#         except json.JSONDecodeError:
#             print("⚠️ Warning: cinema.json is empty or corrupted. Starting fresh.")
#             cinemas_l = []
# else:
#     print("ℹ️ No existing cinema.json found. Starting fresh.")


for url in movie_links:
    print(url)
    movie, cinema_t = get_movie_theater_times(url)
    movies_l.append(movie)
    cinemas_l.append(cinema_t)

https://www.athinorama.gr/cinema/movie/to_47-10086638
WestCity Cinemas did not find cinema info
Δημ. Κιν. Όνειρο Ρέντη did not find cinema info
https://www.athinorama.gr/cinema/movie/agapi_mono-10087322
Αθηνά did not find cinema info
https://www.athinorama.gr/cinema/movie/alpeis-10009258
Studio new star art cinema did not find cinema info
https://www.athinorama.gr/cinema/movie/amugdalia-10068525
Studio new star art cinema did not find cinema info
https://www.athinorama.gr/cinema/movie/o_anthropos_me_to_xruso_xeri-10048555
Studio new star art cinema did not find cinema info
https://www.athinorama.gr/cinema/movie/o_anthropos_xoris_parelthon-1006824
Cinobo Πατησίων did not find cinema info
https://www.athinorama.gr/cinema/movie/o_astakos-10049281
Studio new star art cinema did not find cinema info
https://www.athinorama.gr/cinema/movie/axinos-10087613
Cinobo Όπερα did not find cinema info
Ταινιοθήκη της Ελλάδος did not find cinema info
Μικρόκοσμος did not find cinema info
Δαναός did not f

In [23]:
with open("cinemas.json", "w", encoding="utf-8") as f:
    json.dump(cinemas_l, f, ensure_ascii=False, indent=2)

In [24]:
with open("movies.json", "w", encoding="utf-8") as f:
    json.dump(movies_l, f, ensure_ascii=False, indent=2)

In [ ]:
### PROXEIRO ####

In [ ]:
# import requests


# def get_cinema_info_from_google(name: str, address: str = None):
#     """Fetch cinema info (lat, lon, area, formatted address) from Google Maps API."""
#     query = name if not address else f"{name}, {address}"

#     url = "https://maps.googleapis.com/maps/api/geocode/json"
#     params = {
#         "address": query,
#         "key": GOOGLE_API_KEY,
#         "language": "el"  # or "en" depending on what you want
#     }

#     response = requests.get(url, params=params)
#     data = response.json()

#     if data["status"] != "OK" or not data["results"]:
#         print(f"⚠️ Google Maps API: No match for '{query}'")
#         return {
#             "lat": None,
#             "lon": None,
#             "area": "Unknown",
#             "formatted_address": None
#         }

#     result = data["results"][0]
#     geometry = result["geometry"]["location"]
#     address_components = result.get("address_components", [])

#     # Try to extract area (e.g., neighborhood, locality, sublocality)
#     area = "Unknown"
#     # Extract broader area (default: locality)
#     area = next(
#         (c["long_name"] for c in address_components if "locality" in c["types"]),
#         "Unknown"
#     )

#     open_info = geocode_area(result.get("formatted_address"))
    
#     return {
#         "lat": geometry["lat"],
#         "lon": geometry["lng"],
#         "area": area,
#         "subarea": open_info['area'],
#         "formatted_address": result.get("formatted_address")
#     }

In [ ]:
# get_cinema_info_from_google('Αβάνα',"Λυκούργου 3, 234, Χαλάνδρι 152 31, Ελλάδα")

In [ ]:
# geocode_area('Λυκούργου 3, Χαλάνδρι 152 31, Ελλάδα')

In [ ]:
#     # --- Geocoding ---
#     url = "https://nominatim.openstreetmap.org/search"
#     params = {
#         "q": f"{'Λυκούργου 3, Χαλάνδρι 152 31, Ελλάδα'}",
#         "format": "json",
#         "addressdetails": 1,
#         "limit": 1
#     }

# #     print(f"Geocoding query: {params['q']}")

    
#     r = requests.get(url, params=params, headers={"User-Agent": "cinema-app"})
#     r.raise_for_status()
#     data = r.json()

In [ ]:
# def geocode_area(address):
#     # 🧹 Step 1: Remove parentheses and contents inside them
#     cleaned = re.sub(r'\([^)]*\)', '', address).strip()

#     # 🧹 Step 2: Keep only the first comma-separated part
#     first_part = cleaned.split(',')[0].strip()

#     # 🧹 Step 3: Remove Greek street words and abbreviations
#     first_part = re.sub(
#         r'\b(Λεωφόρος|Λεωφ\.?|Οδός|Οδ\.?|Δρόμος|Δρ\.?)\b',
#         '',
#         first_part,
#         flags=re.IGNORECASE
#     ).strip()

#     # 🧹 Step 4: Keep only up to the first '&' or 'και' or '-' (e.g., "Συγγρού & Φραντζή" → "Συγγρού")
#     first_part = re.split(r'\s*&\s*|\s*και\s*|\s*-\s*', first_part)[0].strip()

#     # 🧹 Step 5: Keep only first word and possible number (e.g. "Παπανδρέου 12")
#     match = re.match(r'^([\wΆ-ώΑ-Ωά-ώ]+(?:\s*\d{1,3})?)', first_part)
#     if match:
#         query_base = match.group(1)
#     else:
#         query_base = first_part

#     # 🧹 Step 6: Collapse spaces
#     query_base = re.sub(r'\s+', ' ', query_base).strip()

#     # --- Geocoding ---
#     url = "https://nominatim.openstreetmap.org/search"
#     params = {
#         "q": f"{query_base}, Athens, Greece",
#         "format": "json",
#         "addressdetails": 1,
#         "limit": 1
#     }

#     print(f"Geocoding query: {params['q']}")

#     try:
#         r = requests.get(url, params=params, headers={"User-Agent": "cinema-app"})
#         r.raise_for_status()
#         data = r.json()
#         if not data:
#             return None

#         details = data[0].get("address", {})
#         area = (
#             details.get("suburb")
#             or details.get("neighbourhood")
#             or details.get("city_district")
#             or details.get("town")
#             or details.get("city")
#         )

#         lat = data[0].get("lat")
#         lon = data[0].get("lon")

#         return {"area": area, "lat": lat, "lng": lon}

#     except Exception as e:
#         print(f"Error geocoding {address}: {e}")
#         return None


In [ ]:
# for url in movie_links:
#     print(url)
#     get_movie_theater(url)

In [ ]:
# for url in movie_links:
#     print(url)
#     get_movie_theater_times(url)

In [ ]:
# url = "https://www.athinorama.gr/cinema/movie/avatar_the_way_of_water-10076581"

# # fetch the page
# response = requests.get(url)
# response.raise_for_status()

# soup = BeautifulSoup(response.text, "html.parser")

# # find all theater blocks
# theaters = []
# for title_tag, details_tag in zip(
#     soup.find_all("h2", class_="item-title"),
#     soup.find_all("div", class_="details")
# ):
#     name = title_tag.get_text(strip=True)
#     address = details_tag.get_text(" ", strip=True)  # keep spacing
#     theaters.append({"name": name, "address": address})

# # print results
# for t in theaters:
#     print(f"{t['name']} - {t['address']}")


In [ ]:
# import requests
# from bs4 import BeautifulSoup

# response = requests.get(url)
# response.raise_for_status()
# soup = BeautifulSoup(response.text, "html.parser")

# cinemas_data = []

# # loop through each cinema entry
# cinema_blocks = soup.find_all("div", class_="item card-item")
# for block in cinema_blocks:
#     # get cinema name and address
#     name_tag = block.find("h2", class_="item-title")
#     details_tag = block.find("div", class_="details")
#     name = name_tag.get_text(strip=True) if name_tag else None
#     address = details_tag.get_text(" ", strip=True) if details_tag else None

#     rooms = []
#     # find panel-inner (each one usually corresponds to a room / screen)
#     for panel in block.find_all("div", class_="grid schedule-grid"):
#         # room name (if available, otherwise fallback)
#         room_name_tag = panel.find("span")
#         room_name = room_name_tag.get_text(strip=True) if room_name_tag else "Main Room"

#         rooms.append({
#             "room": room_name
#         })
        
#     # Find all inner-panel divs
#     room_timetable = []
#     innerpanels = block.find_all("div", class_="panel-inner")
#     for panel in innerpanels:
#             # Get all elements with class daytimeschedule inside each inner-panel
#             schedules = panel.find_all(class_="daytimeschedule")
#             times = [s.get_text(strip=True) for s in schedules]
#             if times:
#                 room_timetable.append(times)        

#     cinemas_data.append({
#         "cinema": name,
#         "address": address,
#         "rooms": rooms,
#         "timetable": room_timetable
#     })

# # show results
# import json
# print(json.dumps(cinemas_data, indent=2, ensure_ascii=False))
